### To have communication V2X we'll use MQTT (GRAY JETBOT)

In [ ]:
# Install MQTT
# !pip3 install paho-mqtt

# DON'T FORGET ACTIVATE THE MQTT BROKER

In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
import paho.mqtt.client as mqtt
import json
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg, Robot
from IPython.display import display
import time  # Para enviar mensajes periódicamente

# Configurar el dispositivo CUDA
device = torch.device('cuda')

#############################################
# Load the TRT optimized model
#############################################

from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth'))
print("Loaded optimized trained weights")

#############################################
# Creating the Pre-Processing Function
#############################################

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

print("Created the Pre-Processing Function")

#############################################
# Initialize Camera
#############################################

camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)
print("Camera displayed")

#############################################
# Initialize JetBot
#############################################

robot = Robot()
print("Created robot instance")

#############################################
# MQTT Configuration (Transmitter)
#############################################

MQTT_BROKER = "192.168.21.1"  # IP del receptor JetBot
MQTT_TOPIC = "jetbot/control"

client = mqtt.Client()
client.connect(MQTT_BROKER)
client.loop_start()

print("MQTT Client Connected and Ready to Publish")

#############################################
# Define Sliders to Control JetBot
#############################################

speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)
print("Created sliders to control JetBot")

#############################################
# Display JetBot Predictions
#############################################

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)
print("Created sliders to see what JetBot is thinking")

#############################################
# Function to Publish Slider Values via MQTT
#############################################

def send_control_data(change=None):
    """Envía los valores actuales de los sliders al receptor JetBot vía MQTT."""
    data = {
        "speed_gain": speed_gain_slider.value,
        "steering_gain": steering_gain_slider.value,
        "steering_kd": steering_dgain_slider.value,
        "steering_bias": steering_bias_slider.value
    }
    
    client.publish(MQTT_TOPIC, json.dumps(data))
    print(f"Sent control data: {data}")

# Vincular los sliders a la función de publicación
speed_gain_slider.observe(send_control_data, names='value')
steering_gain_slider.observe(send_control_data, names='value')
steering_dgain_slider.observe(send_control_data, names='value')
steering_bias_slider.observe(send_control_data, names='value')

#############################################
# Control Loop Function
#############################################

angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    send_control_data()  # Enviar los datos cada vez que se ejecuta el control

execute({'new': camera.value})

#############################################
# Attach Neural Network Execution to Camera
#############################################

camera.observe(execute, names='value')
print("JetBot running...")


In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

camera.stop()

print("Jetbot stopped...")